# DIST-ALERT V1 validation

#### Summary results
DIST-ALERT_v1
For each observed date through the time-series, no-dist and ongoing-dist are compared against the reference time-series for the previous 30 days and finished-dist are compared against the reference time-series year to date. A disturbance must persist in the reference data for at least 15 days (this defends well with the map moving window being +-15 days).

- High magnitude loss for both map and reference (Map ≥50%, ref VLmaj): Users 62.9 ±50.8, Producers 54.2±16.1
    - only confirmed: users 72.8 ±22.6, producers 53.7 ±16.2
- Map high loss, ref any loss: users 81.4 ±62.2
    - only confirmed: **users 94.5 ±11.7**
    - only confirmed, reference lookback of 60 days: users 98.6 ±5.2
    - only confirmed, reference lookback of 60 days, duration 5: **users 99.4 ±1.9**
    - only confirmed, reference lookback of 60 days, duration 10: users 99.2 ±3.1
- Map any loss, ref high loss: **producers 89.4 ±7.8**
    - only confirmed: producers 87.6 ±8.4
    - Lookback 1 day: **producers 91.0 ±7.8**
    - Only confirmed, lookback 1 day: producers 89.5 ±8.3
- Map any loss, ref any loss: users 42.9 ±21.9 prod 63.6 ±26.1
    - Only confirmed: users 70.5 ±10.7 prod 59.0 ±25.2

All of these variations have overall accuracy >98%. For the variations where only users or producers are listed, the other is abysmal (almost by definition).

This means that 63% of high loss alerts were also high loss in the reference with an additional 19% having loss labeled “minority” in the reference. For only confirmed this is 73% and an additional 22% as “minority”. Out of all the “majority” loss identified in the reference, 89% was detected by the product with 88% reaching confirmed status.


### Import strata and sample unit information

In [1]:
# Import strata and sample unit information
import sys 
import math
import datetime

mapsource = "mapLabelsv1sample"
ANNname = "v1sample"
sampleDict = {}
sampleFull = {}
strata = {}
with open("sampledpixels1214.csv",'r') as sample:
  lines = sample.readlines()[1:]
  for l in lines:
    (ID,Stratum,Long,Lat,zone,x,y,pixel,pline,centxUTM,centyUTM,MGRS) = l.strip().split(',')
    sampleDict[ID] = ','.join([Long,Lat,zone,centxUTM,centyUTM])
    sampleFull[ID] = l.strip()
    strata[ID] = int(Stratum)
ids = sampleDict.keys()

#missingBaseline = []#[3,28,87,113,138,139,154,167,205,245]
#missingBaseline_ve = []#[3,28,113,138,139,205,245]
#badQA = [41,85,147]
#refVLmaj = []#[13]

#Strata area
strataAreas = {}
strataCounts = {}
with open("stratatable_0119_z.txt",'r') as file:
  lines = file.readlines()[1:]
for l in lines:
  (s,count,area,K,zcount) = l.strip().split('\t')
  strataAreas[s] = float(area)
  strataCounts[s] = int(zcount)

### Functions to read in map and reference data and calculate accuracy

In [2]:
def getDISTALERTStatus_vI(skipNodata=False):
    mapalert = {}
    for ID in ids:
        mapalert[ID] = [255 for i in range(0,366)]
        with open(mapsource+'/'+ID+'_DIST-ALERT_'+ANNname+'.csv','r') as mapfile:
            lines = mapfile.readlines()
            header = lines[0]
            maplist = lines[1:]
            for line in maplist:
                (granuleID,SensingTime,ProductionTime,VEGDISTSTATUS,VEGANOM,VEGIND,VEGHIST,VEGANOMMAX,VEGDISTCONF,VEGDISTDATE,VEGDISTCOUNT,VEGDISTDUR,VEGLASTDATE,GENDISTSTATUS,GENANOM,GENANOMMAX,GENDISTCONF,GENDISTDATE,GENDISTCOUNT,GENDISTDUR,GENLASTDATE)= line.strip().split(',')
                day = dayDiff("20211001",SensingTime)
                
                if VEGIND!=255:#not (skipNodata and VEGIND==255):
                    if int(VEGDISTSTATUS) in [1,2,3,7] and int(VEGANOMMAX) < 10:
                        mapalert[ID][day] = 0
                    else:
                        mapalert[ID][day] = int(VEGDISTSTATUS)

    return mapalert

In [3]:
def dayDiff(start,end):
  startdate = datetime.datetime.strptime(start,"%Y%m%d")
  enddate = datetime.datetime.strptime(end,"%Y%m%d")
  days = enddate-startdate
  return (days.days+1)

In [4]:
def getRefALERTbinaryDaily(yeslabels,nolabels):
  refalert = {}
  with open('referenceTimeSeries_last.csv','r') as mapfile:
    lines = mapfile.readlines()
    header = lines[0]
    reflist = lines[1:]
  for line in reflist:
    fields = line.strip().split(',')
    (ID,overallLabel,Long,Lat,changetype) = fields[0:5]
    daily = fields[5:]
    refalert[ID] = [0 for i in range(0,365)]
    for day in range(0,365):
      found = False
      for l in yeslabels:
        if l == daily[day]:
          found = True
        #if l in daily[day] and int(ID) in refVLmaj:
        #  found = True
      if found:
        refalert[ID][day] = 2
      else:
        for l in nolabels:
          if l == daily[day]:
            found = True
        if found:
          refalert[ID][day] = 1
        else:
          refalert[ID][day] = 0
  return refalert

In [5]:
def alertConfusionMatrix_vTS2(cat,map,ref,strataCounts,mincount,excludelist,Ndays,name,printMatrix = False):
  N = strataCounts
  Nstrata = len(strataCounts)
  Ntotal = sum([N[str(s)] for s in range(1,Nstrata)])
  n = [[[0,0,0],[0,0,0],[0,0,0]] for s in range(Nstrata+1)]
  ntotal = [0 for s in range(Nstrata+1)]

  #enum Status {NODIST=0,FIRSTLO=1, PROVLO=2,CONFLO=3,FIRSTHI=4,PROVHI=5,CONFHI=6,CONFLOFIN=7,CONFHIFIN=8,NODATA=255};

  if cat == "lt50":
        nodist = [0]
        dist = [1,2,3,4,5,6]
        old = [7,8]
  if cat == "gt50":
        nodist = [0,1,2,3,7]
        dist = [4,5,6]
        old = [8]
  if cat == "confgt50":
        nodist = [0,1,2,3,7]
        dist = [6]
        old = [8]
  if cat == "conflt50":
        nodist = [0]
        dist = [3,6]
        old = [7,8]
  #confusion matrix
  for ID in ids:
    p = [[0,0,0],[0,0,0],[0,0,0]]
    ptotal = 0
    for d in range(0,365):
      #print(ref[ID][d], map[ID][d])
      if not int(ID) in excludelist:
        if max(ref[ID][0:(d+1)])>0 and map[ID][d] != 255:
            start = (d>Ndays)*(d-Ndays)
            if map[ID][d] in nodist:
                mapVal=1
                if ref[ID][start:(d+mincount)].count(2) > mincount:
                    refVal=2
                elif ref[ID][start:(d+1)].count(1) > 0:
                    refVal=1
                else:
                    refVal=0
            elif map[ID][d] in dist:
                mapVal=2
                if ref[ID][start:(d+mincount)].count(2) > mincount:
                    refVal=2
                elif ref[ID][start:(d+1)].count(1) > 0:
                    refVal=1
                else:
                    refVal=0
            elif map[ID][d] in old:
                mapVal=2
                if ref[ID][0:(d+mincount)].count(2) > mincount:
                    refVal=2
                else:
                    refVal=1
            else:
                mapVal=0
                refVal=0
            if refVal>0 and mapVal>0:
                p[refVal][mapVal] += 1
                ptotal += 1
            #if refVal != mapVal and refVal>0 and mapVal>0:
            #    print(ID, d, map[ID][d],mapVal, refVal,ref[ID][start:(d+1)])
    if ptotal>0:
      #if p[1][2]/ptotal >0 and p[2][2]==0:#or p[2][1]/ptotal >0
      #  print(ID, strata[ID], "true: ",round(p[2][2]/ptotal,3),"comm: ", round(p[1][2]/ptotal,3), "om: ", round(p[2][1]/ptotal,3))
      with open(name+".txt",'a') as OUT:
        OUT.write(str(ID)+","+str(p[1][1])+","+str(p[1][2])+","+str(p[2][1])+","+str(p[2][2])+","+str(ptotal)+"\n")
      ntotal[strata[ID]] += (p[1][1]+p[1][2]+p[2][1]+p[2][2])/ptotal
      for r in [1,2]:
        for m in [1,2]:
          n[strata[ID]][r][m] += (p[r][m]/ptotal)
    #print(ptotal,end=",")
  if printMatrix:
    print("r1m1,r1m2,r2m1,r2m2")
    for s in range(1,Nstrata):
      for r in (1,2):
        for m in (1,2):
          print(n[s][r][m],end=",")
      print()
  return (n,ntotal)

In [6]:
def accuracy(n, ntotal, strataCounts,name,write=True):
  N = strataCounts
  Nstrata = len(strataCounts)
  Ntotal = sum([N[str(s)] for s in range(1,Nstrata)])

  #Accuracy
  overall = 0
  y = 0
  usersx = 0
  producersx = 0
  OAsub = 0
  area = [[0,0],[0,0]]
  for s in range(1,Nstrata):
    #overall
    overall += ((n[s][1][1] + n[s][2][2])/ntotal[s])*(N[str(s)]/Ntotal)
    oyhmean = (n[s][1][1] + n[s][2][2])/ntotal[s]
    sampvaryhOA = ((n[s][1][1] + n[s][2][2])*((1-oyhmean)**2) + (n[s][1][2] + n[s][2][1])*((0-oyhmean)**2))/(ntotal[s]-1)
    OAsub += N[str(s)]*N[str(s)]*(1-ntotal[s]/N[str(s)])*sampvaryhOA/(ntotal[s])*(1/Ntotal**2)

    #users and producers
    y += (n[s][2][2]/ntotal[s])*N[str(s)]
    usersx += ((n[s][1][2]+n[s][2][2])/ntotal[s])*N[str(s)]
    producersx += ((n[s][2][1]+n[s][2][2])/ntotal[s])*N[str(s)]
    
    #print("strata",s,n[s][1][1],n[s][1][2],n[s][2][1],n[s][2][2])
    for i in range(1,3):
      for j in range(1,3):
        area[i-1][j-1] += (n[s][i][j]/ntotal[s])*(N[str(s)]/Ntotal)
  
  print(area[0][0]*100,area[0][1]*100)
  print(area[1][0]*100,area[1][1]*100)

  overall = overall*100
  overallSE = math.sqrt(OAsub)*100
  if usersx > 0:
    users = (y/usersx)
  else:
    users = "NA"
    usersSE = "NA"
  if producersx > 0:
    producers = (y/producersx)
  else:
    producers = "NA"
    producersSE = "NA"

  UAsub1 = 0
  UAsub2 = 0
  PAsub1 = 0
  PAsub2 = 0
  for s in range(1,Nstrata):
    #users and producers
    yhmean = n[s][2][2]/ntotal[s]
    yhsampvar = ((n[s][2][2])*((1-yhmean)**2) + (n[s][1][1] + n[s][1][2] + n[s][2][1])*((0-yhmean)**2))/(ntotal[s]-1)
    
    if users != "NA":
        if (n[s][1][2]+n[s][2][2]) > 0:
            xuhmean = (n[s][1][2]+n[s][2][2])/ntotal[s]
            xuhsampvar = ((n[s][1][2]+n[s][2][2])*((1-xuhmean)**2) + (n[s][1][1] + n[s][2][1])*((0-xuhmean)**2))/(ntotal[s]-1)
            xyuhsampvar = (n[s][1][1] * (0-yhmean) * (0-xuhmean) + n[s][1][2] * (0-yhmean) * (1-xuhmean) + n[s][2][1] * (0-yhmean) * (0-xuhmean) + n[s][2][2] * (1-yhmean) * (1-xuhmean))/(ntotal[s] - 1)
            UAsub1 += N[str(s)]*xuhmean
            UAsub2 += N[str(s)]**2 * (1 - ntotal[s]/N[str(s)]) * (yhsampvar + (users**2)*xuhsampvar - 2*users*xyuhsampvar)/ntotal[s]
    
    if producers != "NA":
        if (n[s][2][1]+n[s][2][2]) > 0:
            xphmean = (n[s][2][1]+n[s][2][2])/ntotal[s]
            xphsampvar = ((n[s][2][1]+n[s][2][2])*((1-xphmean)**2) + (n[s][1][1] + n[s][1][2])*((0-xphmean)**2))/(ntotal[s]-1)
            xyphsampvar = (n[s][1][1] * (0-yhmean) * (0-xphmean) + n[s][1][2] * (0-yhmean) * (0-xphmean) + n[s][2][1] * (0-yhmean) * (1-xphmean) + n[s][2][2] * (1-yhmean) * (1-xphmean))/(ntotal[s] - 1)
            PAsub1 += N[str(s)]*xphmean
            PAsub2 += N[str(s)]**2 * (1 - ntotal[s]/N[str(s)]) * (yhsampvar + (producers**2)*xphsampvar - 2*producers*xyphsampvar)/ntotal[s]

  
  if users != "NA":
    usersSE = math.sqrt(1/(UAsub1**2) * UAsub2) * 100
    users = users*100
  else:
    usersSE = "NA"
  if producers != "NA":
    producersSE = math.sqrt(1/(PAsub1**2) * PAsub2) * 100
    producers = producers*100
  else:
    producersSE = "NA"
  print("Overall:",overall," +-", overallSE)
  print("Users:",users," +-", usersSE)
  print("Producers:",producers," +-", producersSE)
  if write:
    with open("accuracy.csv",'a') as OUT:
        OUT.write(','.join([name,str(overall),str(overallSE),str(users),str(usersSE),str(producers),str(producersSE)])+"\n")


## Calculate accuracy under different filtering/grouping rules

In [8]:
#strata enum {CONFHIVEG=6, PROVHIVEG=5, CONFLOWVEG=4,PROVLOWVEG=3,GENDIST=2,NODIST=1,NODATA=0};
mapsource = "mapLabelsv1sample"
for cattype in ["gt50","lt50","confgt50","conflt50"]:
    basename = "v1sample_"+cattype
    map = getDISTALERTStatus_vI(True)
    for duration in [15]:#,5,10,15]:
      for lookback in [30]:#1,15,30]:
        name = basename+"_lookback"+str(lookback)+"_duration"+str(duration)
        noLabels = ["OCmin","OCmaj","OCtotal","noChange","VLmin"]
        ref = getRefALERTbinaryDaily(["VLmaj","VLtotal"],noLabels)
        #with open("ref_accuracy_V1.txt","w") as OUT:
        #    OUT.write(str(ref))
        print("\n"+name)
        (n,ntotal) = alertConfusionMatrix_vTS2(cattype,map,ref,strataCounts,duration,[],lookback,name,False)
        accuracy(n,ntotal,strataCounts,name,True)
        
    for duration in [15]:#,5,10,15]:
      for lookback in [30]:#1,15,30]:
        name = basename+"_lookback"+str(lookback)+"_duration"+str(duration)+"_VLmin"
        noLabels = ["OCmin","OCmaj","OCtotal","noChange"]
        ref = getRefALERTbinaryDaily(["VLmaj","VLtotal","VLmin"],noLabels)
        print("\n"+name)
        (n,ntotal) = alertConfusionMatrix_vTS2(cattype,map,ref,strataCounts,duration,[],lookback,name)
        accuracy(n,ntotal,strataCounts,name,True)


v1sample_gt50_lookback30_duration15
99.60551209100817 0.09549820585979714
0.13687578445634485 0.1621139186756806
Overall: 99.76762600968385  +- 0.21377462381857443
Users: 62.92945992662494  +- 50.81634450868905
Producers: 54.220569129130034  +- 16.10703389607366

v1sample_gt50_lookback30_duration15_VLmin
98.69627974088895 0.048010001291325405
1.0459684410704762 0.20974181674925746
Overall: 98.9060215576382  +- 0.680479970411641
Users: 81.37355474103146  +- 62.15662184094969
Producers: 16.703042397171167  +- 10.56300175800663

v1sample_lt50_lookback30_duration15
97.80032208582209 1.8622470028715736
0.03580396263967903 0.3016269486666684
Overall: 98.10194903448875  +- 1.0437803471790994
Users: 13.939210666695697  +- 7.762483954701868
Producers: 89.38924637903928  +- 7.844494291040573

v1sample_lt50_lookback30_duration15_VLmin
97.3124724376555 1.2307202837895808
0.5301561481848757 0.926651130370045
Overall: 98.23912356802553  +- 1.1288834416888043
Users: 42.952786167837914  +- 21.9453415

______________________________________________________________________________________________________________
### Funtions to calculate accuracy for different subsets of the data using a confidence threshold

In [9]:
def alertConfusionMatrix_varthresh(thresh,var,cat,map,ref,strataCounts,mincount,excludelist,Ndays,printMatrix = False):
  N = strataCounts
  Nstrata = len(strataCounts)
  Ntotal = sum([N[str(s)] for s in range(1,Nstrata)])
  n = [[[0,0,0],[0,0,0],[0,0,0]] for s in range(Nstrata+1)]
  ntotal = [0 for s in range(Nstrata+1)]
  
  if cat == "lt50":
        nodist = [0]
        dist = [1,2,3,4,5,6]
        old = [7,8]
  if cat == "onlylt50":
        nodist = [0]
        dist = [1,2,3]
        old = [7]
  if cat == "gt50":
        nodist = [0,1,2,3,7]
        dist = [4,5,6]
        old = [8]
  if cat == "confgt50":
        nodist = [0,1,2,3,7]
        dist = [6]
        old = [8]
  if cat == "conflt50":
        nodist = [0]
        dist = [3,6]
        old = [7,8]
  #confusion matrix
  for ID in ids:
    p = [[0,0,0],[0,0,0],[0,0,0]]
    ptotal = 0
    for d in range(0,365):
      #print(ref[ID][d], map[ID][d])
      if not int(ID) in excludelist:
        if max(ref[ID][0:(d+1)])>0 and map[ID][d] != 255:
            start = (d>Ndays)*(d-Ndays)
            #Map no disturbance, ref check within lookback to current day plus duration
            if map[ID][d] in nodist:
                mapVal=1
                if ref[ID][start:(d+mincount)].count(2) > mincount:
                    refVal=2
                elif ref[ID][start:(d+1)].count(1) > 0:
                    refVal=1
                else:
                    refVal=0
            #Map disturbance and above variable threshold, ref check within lookback to current day plus duration
            elif map[ID][d] in dist and var[ID][d] >= thresh:
                mapVal=2
                if ref[ID][start:(d+mincount)].count(2) > mincount:
                    refVal=2
                elif ref[ID][start:(d+1)].count(1) > 0:
                    refVal=1
                else:
                    refVal=0
            #Map old disturbance, ref check within start of year to current day plus duration
            elif map[ID][d] in old and var[ID][d] >= thresh:
                mapVal=2
                if ref[ID][0:(d+mincount)].count(2) > mincount:
                    refVal=2
                else:
                    refVal=1
            #Map some kind of disturbance but variable less than threshold so "no", ref check within lookback to current day plus duration
            else:
                mapVal=1
                if ref[ID][start:(d+mincount)].count(2) > mincount:
                    refVal=2
                elif ref[ID][start:(d+1)].count(1) > 0:
                    refVal=1
                else:
                    refVal=0
            if refVal>0 and mapVal>0:
                p[refVal][mapVal] += 1
                ptotal += 1
            #if refVal != mapVal and refVal>0 and mapVal>0:
            #    print(ID, d, map[ID][d],mapVal, refVal,ref[ID][start:(d+1)])
    if ptotal>0:
      #if p[1][2]/ptotal >0 and p[2][2]==0:#or p[2][1]/ptotal >0
      #  print(ID, strata[ID], "true: ",round(p[2][2]/ptotal,3),"comm: ", round(p[1][2]/ptotal,3), "om: ", round(p[2][1]/ptotal,3))
      #if p[2][2]>0:
      #  p[1][2]=0
      #  p[2][1]=0
      ntotal[strata[ID]] += (p[1][1]+p[1][2]+p[2][1]+p[2][2])/ptotal
      for r in [1,2]:
        for m in [1,2]:
          n[strata[ID]][r][m] += (p[r][m]/ptotal)
    #print(ptotal,end=",")
  if printMatrix:
    print("r1m1,r1m2,r2m1,r2m2")
    for s in range(1,Nstrata):
      for r in (1,2):
        for m in (1,2):
          print(n[s][r][m],end=",")
      print()
  return (n,ntotal)

In [10]:
def getDISTALERT_vI_var(variable):
    mapalert = {}
    for ID in ids:
        mapalert[ID] = [255 for i in range(0,366)]
        with open(mapsource+'/'+ID+'_DIST-ALERT_'+ANNname+'.csv','r') as mapfile:
            lines = mapfile.readlines()
            header = lines[0]
            maplist = lines[1:]
            for line in maplist:
                (granuleID,SensingTime,ProductionTime,VEGDISTSTATUS,VEGANOM,VEGIND,VEGHIST,VEGANOMMAX,VEGDISTCONF,VEGDISTDATE,VEGDISTCOUNT,VEGDISTDUR,VEGLASTDATE,GENDISTSTATUS,GENANOM,GENANOMMAX,GENDISTCONF,GENDISTDATE,GENDISTCOUNT,GENDISTDUR,GENLASTDATE)= line.strip().split(',')
                day = dayDiff("20211001",SensingTime)

                if variable == "conf":
                    mapalert[ID][day] = int(VEGDISTCONF)
                elif variable == "dur":
                    mapalert[ID][day] = int(VEGDISTDUR)
                elif variable == "count":
                    mapalert[ID][day] = int(VEGDISTCOUNT)
                elif variable == "maxanom":
                    mapalert[ID][day] = int(VEGANOMMAX)
                else:
                    print("bad variable name")

    return mapalert

_______________________________________________________________________
#### User's accuracy for alerts with increasing confidence thresholds

In [13]:
#User's accuraccy for increasing confidence 
for cattype in ["gt50"]:#map ≥50% loss
    basename = "v1sample_"+cattype
    map = getDISTALERTStatus_vI(True)
    conf = getDISTALERT_vI_var("conf")
    
    for duration in [15]:
      for lookback in [30]:
        for confThresh in [1,50,100,200,300,400,600,800,1000]:
            name = basename+"_conf"+str(confThresh)+"_lookback"+str(lookback)+"_duration"+str(duration)+"_VLmin"
            noLabels = ["OCmin","OCmaj","OCtotal","noChange"]
            ref = getRefALERTbinaryDaily(["VLmaj","VLtotal","VLmin"],noLabels)#ref all veg loss
            print("\n"+name)
            (n,ntotal) = alertConfusionMatrix_varthresh(confThresh,conf,cattype,map,ref,strataCounts,duration,[],lookback)
            accuracy(n,ntotal,strataCounts,name)


v1sample_gt50_conf1_lookback30_duration15_VLmin
98.69627974088895 0.048010001291325405
1.0459684410704762 0.20974181674925746
Overall: 98.9060215576382  +- 0.680479970411641
Users: 81.37355474103146  +- 62.15662184094969
Producers: 16.703042397171167  +- 10.56300175800663

v1sample_gt50_conf50_lookback30_duration15_VLmin
98.69627974088895 0.048010001291325405
1.0459684410704762 0.20974181674925746
Overall: 98.9060215576382  +- 0.680479970411641
Users: 81.37355474103146  +- 62.15662184094969
Producers: 16.703042397171167  +- 10.56300175800663

v1sample_gt50_conf100_lookback30_duration15_VLmin
98.73166873056933 0.012621011610947148
1.0479165316399122 0.20779372617982164
Overall: 98.93946245674914  +- 0.6524396329388998
Users: 94.27397108857221  +- 11.6742653933625
Producers: 16.54790385646845  +- 10.491279302953012

v1sample_gt50_conf200_lookback30_duration15_VLmin
98.73238155485808 0.011908187322187732
1.048148245435732 0.20756201238400174
Overall: 98.93994356724207  +- 0.6524308917213

_______________________________________________________________________
#### Producer's accuracy for alerts with increasing confidence thresholds

In [12]:
#Producer's accuracy for increasing confidence
for cattype in ["lt50"]:#map all loss thresholds
    basename = "v1sample_"+cattype
    map = getDISTALERTStatus_vI(True)
    conf = getDISTALERT_vI_var("conf")
    
    for duration in [15]:
      for lookback in [30]:
        for confThresh in [1,50,100,200,300,400,600,800,1000]:
            name = basename+"_conf"+str(confThresh)+"_lookback"+str(lookback)+"_duration"+str(duration)#+"_VLmin"
            noLabels = ["OCmin","OCmaj","OCtotal","noChange","VLmin"]
            ref = getRefALERTbinaryDaily(["VLmaj","VLtotal"],noLabels)#ref only veg maj and veg total
            print("\n"+name)
            (n,ntotal) = alertConfusionMatrix_varthresh(confThresh,conf,cattype,map,ref,strataCounts,duration,[],lookback)
            accuracy(n,ntotal,strataCounts,name)



v1sample_lt50_conf1_lookback30_duration15
97.80032208582209 1.8622470028715736
0.03580396263967903 0.3016269486666684
Overall: 98.10194903448875  +- 1.0437803471790994
Users: 13.939210666695697  +- 7.762483954701868
Producers: 89.38924637903928  +- 7.844494291040573

v1sample_lt50_conf50_lookback30_duration15
98.59357521449714 1.068993874196532
0.04786917367166787 0.28956173763467963
Overall: 98.8831369521318  +- 0.43052054083085844
Users: 21.3139407112217  +- 8.76095419425958
Producers: 85.81363708311588  +- 9.48303998289195

v1sample_lt50_conf100_lookback30_duration15
98.7623519055421 0.900217183151552
0.052459335907560786 0.2849715753987867
Overall: 99.04732348094089  +- 0.23147882053366964
Users: 24.04440417974848  +- 7.7329839461704015
Producers: 84.45331054452984  +- 9.898519476810204

v1sample_lt50_conf200_lookback30_duration15
98.83277649970788 0.8297925889857803
0.06085613630036227 0.27657477500598515
Overall: 99.10935127471386  +- 0.22429640402520126
Users: 24.99845747511074